In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, log_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.special import softmax
import json
from ucimlrepo import fetch_ucirepo

#spambase = fetch_ucirepo(id=94)

# === User configuration ===
FILE_PATH = "/content/personality_dataset.csv"  # Path to your data file (.csv or .data)
TARGET_COLUMN = 'Personality'          # Name of the target column, or None to auto-detect
TEST_SIZE = 0.2                    # Fraction of data to reserve for testing
VAL_SIZE = 0.25                    # Fraction of train+val data to reserve for validation
N_ESTIMATORS = 100                 # Number of trees in each Random Forest
RANDOM_STATE = 42                  # Random seed for reproducibility
TEMPERATURE = 1.0                  # Softmax temperature (<1 -> sharper; >1 -> smoother)
COMPOSITE_WEIGHTS = [1.0, 1.0, 1.0, 1.0]  # [accuracy, precision, recall, f1]


def load_data(file_path):
    """
    Load dataset by file extension. Supports .csv and .data as CSVs.
    """
    ext = os.path.splitext(file_path)[1].lower()
    if ext in ['.csv', '.data']:
        return pd.read_csv(file_path)
    raise ValueError(f"Unsupported file extension: {ext}")


def detect_target_column(df):
    """
    Auto-detect target column: prefers 'target', then 'class', otherwise last column.
    """
    if 'target' in df.columns:
        return 'target'
    if 'class' in df.columns:
        return 'class'
    return df.columns[-1]


def evaluate_random_forests(df, target_column,
                            test_size=TEST_SIZE, val_size=VAL_SIZE,
                            n_estimators=N_ESTIMATORS, random_state=RANDOM_STATE,
                            temperature=TEMPERATURE, composite_weights=COMPOSITE_WEIGHTS):
    """
    Trains one Random Forest and evaluates:
      - Standard RF with majority voting
      - Composite-metric weighted RF using multiple metrics per tree
    Returns metrics for RF, plus train/test splits, label encoder, and trained RF.
    """
    # Prepare features and labels
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Identify categorical columns for one-hot encoding
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns
    numerical_cols = X.select_dtypes(include=np.number).columns

    # Create a preprocessor using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numerical_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
        ],
        remainder='passthrough' # Keep other columns (if any)
    )

    # Encode string labels to integers
    le = LabelEncoder()
    y_enc = le.fit_transform(y)

    # Split into train+val and test
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y_enc, test_size=test_size, random_state=random_state, stratify=y_enc
    )
    # Split train and val
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=val_size, random_state=random_state, stratify=y_train_val
    )

    # Apply preprocessing
    X_train_processed = preprocessor.fit_transform(X_train)
    X_val_processed = preprocessor.transform(X_val)
    X_test_processed = preprocessor.transform(X_test)


    # Train a single Random Forest
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    rf.fit(X_train_processed, y_train)

    # --- Standard evaluation ---
    proba_std = rf.predict_proba(X_test_processed)
    log_std = log_loss(y_test, proba_std)
    y_pred_std = np.argmax(proba_std, axis=1)
    metrics_standard = classification_report(
        le.inverse_transform(y_test),
        le.inverse_transform(y_pred_std),
        output_dict=True
    )

    # --- Composite-metric weighting ---
    # Compute per-tree metrics on validation set
    tree_acc, tree_prec, tree_rec, tree_f1 = [], [], [], []
    for tree in rf.estimators_:
        # Need to pass processed data to the tree
        preds_val = tree.predict(X_val_processed)
        tree_acc.append(accuracy_score(y_val, preds_val))
        tree_prec.append(precision_score(y_val, preds_val, average='weighted', zero_division=0))
        tree_rec.append(recall_score(y_val, preds_val, average='weighted', zero_division=0))
        tree_f1.append(f1_score(y_val, preds_val, average='weighted', zero_division=0))
    metrics_matrix = np.vstack([tree_acc, tree_prec, tree_rec, tree_f1]).T
    composite_scores = metrics_matrix.dot(np.array(composite_weights))
    weights = softmax(composite_scores / temperature)

    # Aggregate weighted probabilities
    proba_weighted = np.zeros_like(proba_std)
    for w, tree in zip(weights, rf.estimators_):
        # Need to pass processed data to the tree
        proba_weighted += w * tree.predict_proba(X_test_processed)
    log_weighted = log_loss(y_test, proba_weighted)
    y_pred_w = np.argmax(proba_weighted, axis=1)
    metrics_weighted = classification_report(
        le.inverse_transform(y_test),
        le.inverse_transform(y_pred_w),
        output_dict=True
    )

    return (metrics_standard, metrics_weighted, log_std, log_weighted,
            composite_scores, weights, X_train_processed, y_train, X_test_processed, y_test, le, rf, preprocessor)


def print_summary(metrics, title, logloss=None):
    """
    Prints overall accuracy, weighted-average precision, recall, F1, and optional log-loss.
    """
    accuracy = metrics.get('accuracy')
    weighted_avg = metrics.get('weighted avg', {})
    precision = weighted_avg.get('precision')
    recall = weighted_avg.get('recall')
    f1 = weighted_avg.get('f1-score')

    print(f"\n{title} Summary:")
    if accuracy is not None:
        print(f"  Accuracy : {accuracy:.4f}")
    if precision is not None:
        print(f"  Precision: {precision:.4f}")
    if recall is not None:
        print(f"  Recall   : {recall:.4f}")
    if f1 is not None:
        print(f"  F1-score : {f1:.4f}")
    if logloss is not None:
        print(f"  Log-loss : {logloss:.4f}")


def main():
    # Load data
    df = load_data(FILE_PATH)
    target_col = TARGET_COLUMN or detect_target_column(df)
    print(f"Auto-detected target column: {target_col}\n")

    # Evaluate Random Forests and get splits
    (metrics_standard, metrics_weighted,
     log_std, log_weighted,
     composite_scores, weights,
     X_train_processed, y_train, X_test_processed, y_test,
     le, rf, preprocessor) = evaluate_random_forests(df, target_col)

    # Full reports for RF models
    print("Standard Random Forest Performance:")
    print(json.dumps(metrics_standard, indent=2))
    print(f"\nStandard RF Log Loss: {log_std:.4f}\n")

    print("Composite-Metric Softmax RF Performance:")
    print(json.dumps(metrics_weighted, indent=2))
    print(f"\nComposite Softmax RF Log Loss: {log_weighted:.4f}\n")

    # Display composite scores and weight distribution
    print(f"Composite Scores (first 10 trees): {np.round(composite_scores[:10], 4)}")
    print(f"Softmax Weights   (first 10 trees): {np.round(weights[:10], 4)}")

    # Concise summaries including log-loss
    print_summary(metrics_standard, "Standard Random Forest", log_std)
    print_summary(metrics_weighted, "Composite-Metric RF (softmax)", log_weighted)

    # === Stacked MOE Model Evaluation ===
    # Define a simple stacking (MOE) classifier using RF as an expert
    experts = [('rf', rf)]
    stacked_moe = StackingClassifier(
        estimators=experts,
        final_estimator=LogisticRegression(),
        cv=5,
        stack_method='predict_proba',
        passthrough=False,
        n_jobs=-1
    )
    stacked_moe.fit(X_train_processed, y_train)

    # Evaluate the Stacked MOE on the test set
    proba_moe = stacked_moe.predict_proba(X_test_processed)
    log_moe = log_loss(y_test, proba_moe)
    y_pred_moe = np.argmax(proba_moe, axis=1)

    metrics_moe = classification_report(
        le.inverse_transform(y_test),
        le.inverse_transform(y_pred_moe),
        output_dict=True
    )

    print("\nStacked MOE Model Performance:")
    print(json.dumps(metrics_moe, indent=2))
    print(f"\nStacked MOE Model Log Loss: {log_moe:.4f}\n")
    print_summary(metrics_moe, "Stacked MOE Model", log_moe)


if __name__ == "__main__":
    main()

Auto-detected target column: Personality

Standard Random Forest Performance:
{
  "Extrovert": {
    "precision": 0.9143835616438356,
    "recall": 0.8959731543624161,
    "f1-score": 0.9050847457627119,
    "support": 298.0
  },
  "Introvert": {
    "precision": 0.8923611111111112,
    "recall": 0.9113475177304965,
    "f1-score": 0.9017543859649123,
    "support": 282.0
  },
  "accuracy": 0.903448275862069,
  "macro avg": {
    "precision": 0.9033723363774734,
    "recall": 0.9036603360464563,
    "f1-score": 0.9034195658638121,
    "support": 580.0
  },
  "weighted avg": {
    "precision": 0.9036760943158558,
    "recall": 0.903448275862069,
    "f1-score": 0.9034655018610231,
    "support": 580.0
  }
}

Standard RF Log Loss: 0.6166

Composite-Metric Softmax RF Performance:
{
  "Extrovert": {
    "precision": 0.9140893470790378,
    "recall": 0.8926174496644296,
    "f1-score": 0.9032258064516129,
    "support": 298.0
  },
  "Introvert": {
    "precision": 0.889273356401384,
    "re

dw abt this

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, log_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy.special import softmax
import json
from ucimlrepo import fetch_ucirepo

# === User configuration ===
TRAIN_PATH = "/content/train.csv"  # Path to your training data file
TEST_PATH  = "/content/test.csv"   # Path to your test data file
TARGET_COLUMN = 'satisfaction'                    # Name of the target column, or None to auto-detect
VAL_SIZE = 0.25                            # Fraction of TRAIN data to reserve for validation
N_ESTIMATORS = 100                          # Number of trees in each Random Forest
RANDOM_STATE = 42                          # Random seed for reproducibility
TEMPERATURE = 1.0                          # Softmax temperature (<1 -> sharper; >1 -> smoother)
COMPOSITE_WEIGHTS = [1.0, 1.0, 1.0, 1.0]    # [accuracy, precision, recall, f1]


def load_data(file_path):
    """
    Load dataset by file extension. Supports .csv and .data as CSVs.
    """
    ext = os.path.splitext(file_path)[1].lower()
    if ext in ['.csv', '.data']:
        return pd.read_csv(file_path)
    raise ValueError(f"Unsupported file extension: {ext}")


def detect_target_column(df):
    """
    Auto-detect target column: prefers 'target', then 'class', otherwise last column.
    """
    if 'target' in df.columns:
        return 'target'
    if 'class' in df.columns:
        return 'class'
    return df.columns[-1]


def evaluate_random_forests_explicit(df_train, df_test, target_column,
                                    val_size=VAL_SIZE, n_estimators=N_ESTIMATORS,
                                    random_state=RANDOM_STATE, temperature=TEMPERATURE,
                                    composite_weights=COMPOSITE_WEIGHTS):
    """
    Trains one Random Forest on df_train (optionally splitting off validation),
    evaluates on df_test, for both standard majority-vote RF and composite-metric softmax RF.
    Returns metrics, log-losses, composite scores, weights, processed splits, label encoder, RF, and preprocessor.
    """
    # Split out features & labels
    X_train_full = df_train.drop(columns=[target_column])
    y_train_full = df_train[target_column]
    X_test = df_test.drop(columns=[target_column])
    y_test = df_test[target_column]

    # Encode labels to integers
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train_full)
    y_test_enc  = le.transform(y_test)

    # Optionally split train into train + validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_enc, test_size=val_size,
        random_state=random_state, stratify=y_train_enc
    )

    # Identify categorical and numerical columns
    categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
    numerical_cols   = X_train.select_dtypes(include=np.number).columns

    # Build preprocessing pipeline
    preprocessor = ColumnTransformer([
        ('num', 'passthrough', numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ], remainder='passthrough')

    # Fit on train, transform train/val/test
    X_train_proc = preprocessor.fit_transform(X_train)
    X_val_proc   = preprocessor.transform(X_val)
    X_test_proc  = preprocessor.transform(X_test)

    # Train a single Random Forest
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    rf.fit(X_train_proc, y_train)

    # --- Standard RF evaluation ---
    proba_std = rf.predict_proba(X_test_proc)
    log_std = log_loss(y_test_enc, proba_std)
    y_pred_std = np.argmax(proba_std, axis=1)
    metrics_standard = classification_report(
        le.inverse_transform(y_test_enc),
        le.inverse_transform(y_pred_std),
        output_dict=True
    )

    # --- Composite-metric softmax RF ---
    # Compute per-tree metrics on validation set
    tree_acc, tree_prec, tree_rec, tree_f1 = [], [], [], []
    for tree in rf.estimators_:
        preds_val = tree.predict(X_val_proc)
        tree_acc.append(accuracy_score(y_val, preds_val))
        tree_prec.append(precision_score(y_val, preds_val, average='weighted', zero_division=0))
        tree_rec.append(recall_score(y_val, preds_val, average='weighted', zero_division=0))
        tree_f1.append(f1_score(y_val, preds_val, average='weighted', zero_division=0))

    metrics_matrix = np.vstack([tree_acc, tree_prec, tree_rec, tree_f1]).T
    composite_scores = metrics_matrix.dot(np.array(composite_weights))
    weights = softmax(composite_scores / temperature)

    # Aggregate weighted probabilities
    proba_weighted = np.zeros_like(proba_std)
    for w, tree in zip(weights, rf.estimators_):
        proba_weighted += w * tree.predict_proba(X_test_proc)

    log_weighted = log_loss(y_test_enc, proba_weighted)
    y_pred_w = np.argmax(proba_weighted, axis=1)
    metrics_weighted = classification_report(
        le.inverse_transform(y_test_enc),
        le.inverse_transform(y_pred_w),
        output_dict=True
    )

    return (metrics_standard, metrics_weighted,
            log_std, log_weighted,
            composite_scores, weights,
            X_train_proc, y_train, X_test_proc, y_test_enc,
            le, rf, preprocessor)


def print_summary(metrics, title, logloss=None):
    """
    Prints overall accuracy, weighted-average precision, recall, F1, and optional log-loss.
    """
    accuracy = metrics.get('accuracy')
    weighted = metrics.get('weighted avg', {})
    precision = weighted.get('precision')
    recall = weighted.get('recall')
    f1 = weighted.get('f1-score')

    print(f"\n{title} Summary:")
    if accuracy is not None:
        print(f"  Accuracy : {accuracy:.4f}")
    if precision is not None:
        print(f"  Precision: {precision:.4f}")
    if recall is not None:
        print(f"  Recall   : {recall:.4f}")
    if f1 is not None:
        print(f"  F1-score : {f1:.4f}")
    if logloss is not None:
        print(f"  Log-loss : {logloss:.4f}")


def main():
    # Load train and test files
    df_train = load_data(TRAIN_PATH)
    df_test  = load_data(TEST_PATH)

    # Determine target column
    target_col = TARGET_COLUMN or detect_target_column(df_train)
    print(f"Using '{target_col}' as target column.\n")

    # Evaluate Random Forests with explicit train/test
    (metrics_std, metrics_w,
     log_std, log_w,
     comp_scores, weights,
     X_train_p, y_train, X_test_p, y_test_enc,
     le, rf, preprocessor) = evaluate_random_forests_explicit(
         df_train, df_test, target_col
    )

    # Report standard RF
    print("Standard Random Forest Performance:")
    print(json.dumps(metrics_std, indent=2))
    print(f"\nStandard RF Log Loss: {log_std:.4f}")
    print_summary(metrics_std, "Standard RF", log_std)

    # Report composite-metric RF
    print("\nComposite-Metric Softmax RF Performance:")
    print(json.dumps(metrics_w, indent=2))
    print(f"\nComposite Softmax RF Log Loss: {log_w:.4f}")
    print_summary(metrics_w, "Composite-Metric RF (softmax)", log_w)

    # Show example composite scores & weights
    print(f"\nComposite Scores (first 10 trees): {np.round(comp_scores[:10],4)}")
    print(f"Weights            (first 10 trees): {np.round(weights[:10],4)}")

    # === Stacked MOE Model Evaluation ===
    experts = [('rf', rf)]
    stacked_moe = StackingClassifier(
        estimators=experts,
        final_estimator=LogisticRegression(),
        cv=5,
        stack_method='predict_proba',
        passthrough=False,
        n_jobs=-1
    )
    stacked_moe.fit(X_train_p, y_train)

    proba_moe = stacked_moe.predict_proba(X_test_p)
    log_moe = log_loss(y_test_enc, proba_moe)
    y_pred_moe = np.argmax(proba_moe, axis=1)
    metrics_moe = classification_report(
        le.inverse_transform(y_test_enc),
        le.inverse_transform(y_pred_moe),
        output_dict=True
    )

    print("\nStacked MOE Model Performance:")
    print(json.dumps(metrics_moe, indent=2))
    print(f"\nStacked MOE Model Log Loss: {log_moe:.4f}")
    print_summary(metrics_moe, "Stacked MOE Model", log_moe)


if __name__ == "__main__":
    main()


Using 'satisfaction' as target column.

Standard Random Forest Performance:
{
  "neutral or dissatisfied": {
    "precision": 0.957259796027912,
    "recall": 0.9790022644616757,
    "f1-score": 0.9680089561352919,
    "support": 14573.0
  },
  "satisfied": {
    "precision": 0.9723627167630058,
    "recall": 0.9441375076734193,
    "f1-score": 0.9580422691879866,
    "support": 11403.0
  },
  "accuracy": 0.9636972590083154,
  "macro avg": {
    "precision": 0.964811256395459,
    "recall": 0.9615698860675475,
    "f1-score": 0.9630256126616392,
    "support": 25976.0
  },
  "weighted avg": {
    "precision": 0.9638897084525453,
    "recall": 0.9636972590083154,
    "f1-score": 0.9636337585967902,
    "support": 25976.0
  }
}

Standard RF Log Loss: 0.1079

Standard RF Summary:
  Accuracy : 0.9637
  Precision: 0.9639
  Recall   : 0.9637
  F1-score : 0.9636
  Log-loss : 0.1079

Composite-Metric Softmax RF Performance:
{
  "neutral or dissatisfied": {
    "precision": 0.9581317204301075,


dw abt htis either

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    log_loss
)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy.special import softmax
import json
from ucimlrepo import fetch_ucirepo

# Fetch example dataset (unused in this script but kept for context)
spambase = fetch_ucirepo(id=94)

# === User configuration ===
FILE_PATH = "/content/defaultOfCreditCardClients.xls"  # Path to your data file (.csv, .data, .xls, .xlsx)
TARGET_COLUMN = 'Y'          # Name of the target column, or None to auto-detect
TEST_SIZE = 0.2                    # Fraction of data to reserve for testing
VAL_SIZE = 0.25                    # Fraction of train+val data to reserve for validation
N_ESTIMATORS = 30                 # Number of trees in each Random Forest
RANDOM_STATE = 42                  # Random seed for reproducibility
TEMPERATURE = 1.0                  # Softmax temperature (<1 -> sharper; >1 -> smoother)
COMPOSITE_WEIGHTS = [1.0, 1.0, 1.0, 1.0]  # [accuracy, precision, recall, f1]


def load_data(file_path):
    """
    Load dataset by file extension. Supports:
      - .csv / .data via read_csv
      - .xls / .xlsx via read_excel (skipping 2nd row)
    After loading, always drop the first column.
    """
    ext = os.path.splitext(file_path)[1].lower()
    if ext in ['.csv', '.data']:
        df = pd.read_csv(file_path)
    elif ext in ['.xls', '.xlsx']:
        # header=0 → first row as column names; skiprows=[1] → drop 2nd row
        df = pd.read_excel(file_path, header=0, skiprows=[1])
    else:
        raise ValueError(f"Unsupported file extension: {ext}")

    # Drop the first column (position 0) unconditionally
    return df.drop(df.columns[0], axis=1)


def detect_target_column(df):
    """
    Auto-detect target column: prefers 'target', then 'class', otherwise last column.
    """
    if 'target' in df.columns:
        return 'target'
    if 'class' in df.columns:
        return 'class'
    return df.columns[-1]


def evaluate_random_forests(
    df, target_column,
    test_size=TEST_SIZE, val_size=VAL_SIZE,
    n_estimators=N_ESTIMATORS, random_state=RANDOM_STATE,
    temperature=TEMPERATURE, composite_weights=COMPOSITE_WEIGHTS
):
    """
    Trains one Random Forest and evaluates:
      - Standard RF with majority voting
      - Composite-metric weighted RF using multiple metrics per tree
    Returns metrics for RF, plus train/test splits, label encoder, and trained RF.
    """
    # Prepare features and labels
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Identify categorical and numerical columns
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns
    numerical_cols   = X.select_dtypes(include=np.number).columns

    # Build preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numerical_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
        ],
        remainder='passthrough'
    )

    # Encode target labels
    le = LabelEncoder()
    y_enc = le.fit_transform(y)

    # Split into train+val and test
    X_tr_val, X_test, y_tr_val, y_test = train_test_split(
        X, y_enc,
        test_size=test_size,
        random_state=random_state,
        stratify=y_enc
    )
    # Split train and validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr_val, y_tr_val,
        test_size=val_size,
        random_state=random_state,
        stratify=y_tr_val
    )

    # Preprocess features
    X_train_p = preprocessor.fit_transform(X_train)
    X_val_p   = preprocessor.transform(X_val)
    X_test_p  = preprocessor.transform(X_test)

    # Train Random Forest
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        random_state=random_state
    )
    rf.fit(X_train_p, y_train)

    # Standard RF evaluation
    proba_std = rf.predict_proba(X_test_p)
    log_std   = log_loss(y_test, proba_std)
    y_pred_std = np.argmax(proba_std, axis=1)
    metrics_std = classification_report(
        le.inverse_transform(y_test),
        le.inverse_transform(y_pred_std),
        output_dict=True
    )

    # Composite-metric weighted RF
    # Compute per-tree metrics on validation set
    metrics_matrix = []
    for tree in rf.estimators_:
        preds_val = tree.predict(X_val_p)
        acc  = accuracy_score(y_val, preds_val)
        prec = precision_score(y_val, preds_val, average='weighted', zero_division=0)
        rec  = recall_score(y_val, preds_val, average='weighted', zero_division=0)
        f1   = f1_score(y_val, preds_val, average='weighted', zero_division=0)
        metrics_matrix.append([acc, prec, rec, f1])
    metrics_matrix = np.array(metrics_matrix)  # shape=(n_trees, 4)

    # Composite scores and softmax weights
    composite_scores = metrics_matrix.dot(np.array(composite_weights))
    weights = softmax(composite_scores / temperature)

    # Weighted probability aggregation
    proba_w = np.zeros_like(proba_std)
    for w, tree in zip(weights, rf.estimators_):
        proba_w += w * tree.predict_proba(X_test_p)
    log_w = log_loss(y_test, proba_w)
    y_pred_w = np.argmax(proba_w, axis=1)
    metrics_w = classification_report(
        le.inverse_transform(y_test),
        le.inverse_transform(y_pred_w),
        output_dict=True
    )

    return (
        metrics_std, metrics_w,
        log_std, log_w,
        composite_scores, weights,
        X_train_p, y_train,
        X_test_p, y_test,
        le, rf, preprocessor
    )


def print_summary(metrics, title, logloss=None):
    """
    Prints overall accuracy, weighted-average precision, recall, F1, and optional log-loss.
    """
    accuracy     = metrics.get('accuracy')
    weighted_avg = metrics.get('weighted avg', {})
    precision    = weighted_avg.get('precision')
    recall       = weighted_avg.get('recall')
    f1           = weighted_avg.get('f1-score')

    print(f"\n{title} Summary:")
    if accuracy is not None:
        print(f"  Accuracy : {accuracy:.4f}")
    if precision is not None:
        print(f"  Precision: {precision:.4f}")
    if recall is not None:
        print(f"  Recall   : {recall:.4f}")
    if f1 is not None:
        print(f"  F1-score : {f1:.4f}")
    if logloss is not None:
        print(f"  Log-loss : {logloss:.4f}")


def main():
    # Load data (skips 2nd row and drops 1st column for Excel)
    df = load_data(FILE_PATH)
    target_col = TARGET_COLUMN or detect_target_column(df)
    print(f"Auto-detected target column: {target_col}\n")

    # Evaluate Random Forests
    (
        metrics_std, metrics_w,
        log_std, log_w,
        composite_scores, weights,
        X_train_p, y_train,
        X_test_p, y_test,
        le, rf, preprocessor
    ) = evaluate_random_forests(df, target_col)

    # Print full reports
    print("Standard Random Forest Performance:")
    print(json.dumps(metrics_std, indent=2))
    print(f"\nStandard RF Log Loss: {log_std:.4f}\n")

    print("Composite-Metric Softmax RF Performance:")
    print(json.dumps(metrics_w, indent=2))
    print(f"\nComposite Softmax RF Log Loss: {log_w:.4f}\n")

    # Display composite scores and weight distribution
    print(f"Composite Scores (first 10 trees): {np.round(composite_scores[:10], 4)}")
    print(f"Softmax Weights   (first 10 trees): {np.round(weights[:10], 4)}")

    # Concise summaries
    print_summary(metrics_std, "Standard Random Forest", log_std)
    print_summary(metrics_w, "Composite-Metric RF (softmax)", log_w)

    # === Stacked MOE Model Evaluation ===
    experts = [('rf', rf)]
    stacked_moe = StackingClassifier(
        estimators=experts,
        final_estimator=LogisticRegression(),
        cv=5,
        stack_method='predict_proba',
        passthrough=False,
        n_jobs=-1
    )
    stacked_moe.fit(X_train_p, y_train)

    proba_moe = stacked_moe.predict_proba(X_test_p)
    log_moe   = log_loss(y_test, proba_moe)
    y_pred_moe = np.argmax(proba_moe, axis=1)
    metrics_moe = classification_report(
        le.inverse_transform(y_test),
        le.inverse_transform(y_pred_moe),
        output_dict=True
    )

    print("\nStacked MOE Model Performance:")
    print(json.dumps(metrics_moe, indent=2))
    print(f"\nStacked MOE Model Log Loss: {log_moe:.4f}\n")
    print_summary(metrics_moe, "Stacked MOE Model", log_moe)


if __name__ == "__main__":
    main()
